In [ ]:
import pandas as pd
import re
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

df = pd.read_csv("/home/jennifer/Documentos/tercer_año/segundo_semestre/SRI/proyecto final/papers_guardados.csv")  
a=0
def remove_sections(text,urls):
    global a
    a+=1
    after_refs = re.split(r'\n(references|bibliography|acknowledg(e)?ments)\b', text, flags=re.IGNORECASE)  

    url_patron= r'https?://\S+|www\.\S+'

    for i in after_refs:
        url=[]
        if not i:
            continue
        if len(i)==0:
            continue
        url = re.findall(url_patron, i)
        url1 = [re.sub(r'[).,;]+$', '', url_sin_parentesis.strip()) for url_sin_parentesis in url if url_sin_parentesis.strip()]
        if url1:
            for k in url1:
                urls.append(k)
    print(f"Total de papers visitados {a}")
    return after_refs[0]

urls_collected = []
df['clean_contenido'] = df['contenido'].apply(lambda x: remove_sections(x, urls_collected))

# Guardar las URLs en un archivo
urls_df = pd.DataFrame({'URL': urls_collected})
urls_df.to_csv('urls_encontradas.csv', index=False)  # Guardar como CSV
# Alternativa para guardar como TXT:
# with open('urls_encontradas.txt', 'w') as f:
#     f.write('\n'.join(urls_collected))

print(f"Se encontraron y guardaron {len(urls_collected)} URLs.")

In [ ]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PyPDF2 import PdfReader
import time
import re
import os 

VISITED_DOIS_FILE = "dois_visitados.txt"
OUTPUT_FILE = "../proyecto final/papers_guardados.csv"

if os.path.exists(VISITED_DOIS_FILE):
    with open(VISITED_DOIS_FILE, "r") as f:
        existing_dois = set(f.read().splitlines())
else:
    existing_dois = set()

if os.path.exists(OUTPUT_FILE):
    df_existente = pd.read_csv(OUTPUT_FILE)
    dois_guardados = set(df_existente["doi"])
    results = df_existente.to_dict(orient="records")
else:
    df_existente = pd.DataFrame()
    dois_guardados = set()
    results = []
print(results)
# EMAIL = "jennifersanchezsantana36@gemail.com"

def obtener_datos_unpaywall(doi, email):
    url = f"https://api.unpaywall.org/v2/{doi}" 
    params = {
        "email": email  # Es OBLIGATORIO incluir tu email
    }
    try:
        response = requests.get(url, params=params, timeout=15)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.ConnectionError as e:
        print(f"❌ Error de conexión para DOI {doi}: {str(e)}")
        time.sleep(5)  # Espera más ante errores de conexión
        return None
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Error HTTP para DOI {doi}: {str(e)}")
        return None
def extract_text_from_pdf(pdf_url):
    try:
        response = requests.get(pdf_url, timeout=10)
        response.raise_for_status()
        time.sleep(1)

        if "application/pdf" not in response.headers.get("Content-Type", ""):
            return None
        with BytesIO(response.content) as f:
            reader = PdfReader(f)
            return "\n".join(filter(None, (p.extract_text() for p in reader.pages))).strip()
    except Exception as e:
        print(f"❌ Error leyendo PDF: {e}")
        return None
    
def extraer_info_relevante(datos):
    title = datos.get("title", "")
    doi = datos.get("doi", "")
    if not datos:
        return None
    if not datos.get("best_oa_location"):
        return None 
    landing_page = datos.get("best_oa_location", {}).get("url_for_landing_page", "")
    
    # Unir todos los nombres de autores
    authors = datos.get("z_authors", [])
    authors_str = ", ".join([a.get("raw_author_name", "") for a in authors])
    
    published = datos.get("published_date", "")
    language = datos.get("language", "en")
    
    # No siempre está presente
    abstract = datos.get("abstract", "No abstract available.")

    return {
        "title": title,
        "doi": doi,
        "landing_page": landing_page,
        "authors": authors_str,
        "published": published,
        "abstract": abstract,
        "language": language
    }
def find_direct_pdf_link(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        if "text/html" in response.headers.get("Content-Type", ""):
            soup = BeautifulSoup(response.text, "html.parser")
            for link in soup.find_all("a", href=True):
                if ".pdf" in link["href"].lower():
                    return requests.compat.urljoin(url, link["href"])
    except Exception as e:
        print(f"❌ PDF manual falló: {e}")
    return None
def get_open_access_pdf(doi, email=EMAIL):
    url = f"https://api.unpaywall.org/v2/{doi}"
    params = {"email": email}
    try:
        response = requests.get(url, params=params).json()
        if response.get("is_oa"):
            return response.get("best_oa_location", {}).get("url_for_pdf")
    except Exception as e:
        print(f"❌ Error en Unpaywall: {e}")
    return None

def extract_dois_from_text(text):
    # DOI estándar: 10.<registro>/<sufijo>
    doi_pattern = r"10.\d{4,9}/[-._;()/:A-Z0-9]+"
    return re.findall(doi_pattern, text, flags=re.IGNORECASE)
def is_doi(url):
    """Verifica si la URL es un DOI."""
    return "doi.org/10" in url.lower()
a=['https://doi.org/10.12725/ujbm.52.6', 'https://doi.org/10.3233/IDT', 'https://doi.org/10.4236/ojbm.2018.63048', 'https://doi.org/10.21511/im.15(3).2019.04', 'https://doi.org/10.1016/j.ijtst.2019.09.004', 'https://doi.org/10.1016/bs.adcom.2019.10.006', 'https://doi.org/10.1007/s12652', 'https://doi.org/10.1016/j.eswa.2018.08', 'https://doi.org/10.1109/LRA.2018.2800119']

# for url in a:
#     if is_doi:
#         doi=extract_dois_from_text(url)[0]
#         datos = obtener_datos_unpaywall(doi, EMAIL)
        
#         if datos:
#             info = extraer_info_relevante(datos)
#             if not info:
#                 print("❌ Error en url extraida")
#                 continue
#             url_pdf = datos.get("best_oa_location", {}).get("landing_page", "")
#             if not url_pdf:
#                 url_pdf=datos.get("best_oa_location", {}).get("url_for_pdf")
#                 if not url_pdf:
#                     print("⚠️ Url no validada")
#             content = extract_text_from_pdf(url_pdf) if url_pdf else None
#             print(content)
#             time.sleep(1) 
count=0
for url in urls_collected:
    count+=1
    print(count)
    if is_doi(url):
        with open(VISITED_DOIS_FILE, "a") as f_dois:
            doi=extract_dois_from_text(url)[0]
            datos = obtener_datos_unpaywall(doi, EMAIL)
            if datos:
                if doi in  dois_guardados:
                    continue
                paper = extraer_info_relevante(datos)
                print("❌ Error en url extraida")

                if not paper:
                    continue
                doi = paper["doi"]
                existing_dois.add(doi)
                landing_url = paper["landing_page"]
                pdf_url = get_open_access_pdf(doi)
                if not pdf_url:
                    pdf_url = find_direct_pdf_link(landing_url)
                
                paper["pdf_url"] = pdf_url
                content = extract_text_from_pdf(pdf_url) if pdf_url else None
                paper["contenido"] = content
                if content and content!="No disponible":
                    results.add(paper)
                    dois_guardados.append(doi)
                    f_dois.write(doi + "\n")
                    total_collected += 1

                    print(f"✅ ({total_collected}) {paper['title'][:60]}...")
                    
                    pd.DataFrame([paper]).to_csv(OUTPUT_FILE, mode="a", header=not os.path.exists(OUTPUT_FILE), index=False, encoding='utf-8',errors='ignore')
                else:
                    print(f"⚠️ Sin contenido: {paper['title'][:60]}")

                time.sleep(1)  
            